# Handling Exceptions

When there is an error during the execution of a program, Python will raise an _exception_. In general the script will abort and print an error message. A very common error is trying to read a file that doesn't exist.

You can handle an exception and try to save the script.

## Reading JSON

As example of handling an exception, we will try to fix a JSON file that is quoted.

In [1]:
import json as JSON
import logging as LOG

We didn't speak about [logging](https://docs.python.org/3.11/library/logging.html), but it's a very convenient way to control the 'verbosity' of the script: you can set to "info" to see all messages or 'error" to see just the error messages.

> Jupyter users: after changing the level of logging, it's necessary to restart the kernel.

Next we define two functions to handle the JSON data. The first will try to read the file, and, if everything goes well, it will simply return a dictonary `data.` But, on the other hand, there is an error,  then, the function will call another function to try to fix the file.

The second function, `clean_json` will try to open the file as simple _string_, remove the quotes, and return the JSON data.

In the end, if both attempts to clean the JSON, fail, we return an empty dictionary to the main routine.

In [18]:
LOG.basicConfig(encoding="utf-8", level=LOG.INFO)

def clean_json(json_file: str) -> dict:
    """Clean the JSON file by removing the external quotes.
    Returns json data or an empty dict in case unable to clean."""

    LOG.info(f"Trying to clean JSON file '{json_file}'")
    try:
        with open(json_file) as fin:
            in_text = fin.readlines()
        #
        text = str(in_text[0])
        len_text = len(text)
        data = JSON.loads(text[1 : len_text - 1])
    except ValueError as ee:
        LOG.error(f"Unable to clean JSON file '{json_file}'")
        data = {}
    finally:
        return data


def read_json_file(json_file: str) -> dict:
    """Read the JSON file.
    Returns the JSON data or an empty dict in case of error."""

    # Try to read the json
    LOG.info(f"Reading JSON file '{json_file}'")
    try:
        with open(json_file, "r", encoding="utf-8") as fin:
            data = JSON.load(fin)
    except ValueError as ee:
        LOG.warning(f"There was error '{ee}' while reading JSON file '{json_file}'. Trying to fix file.")
        data = clean_json(json_file)
    finally:
        return data

Below is the main routine, and if the `data` is empty, we raise an exception to next level, and if there is none (we are really the main routine), then abort the script.

In [19]:
json_file = "10754_136193.json"
data = read_json_file(json_file)
# data = {}
if data:
    LOG.info("Cleaning successful")
    metadata = data['metadata']
else:
    LOG.critical('Nothing was read from the file. Somethig very wrong!')
    raise ValueError('Metadata is empty after trying to fix it.')

INFO:root:Reading JSON file '10754_136193.json'
INFO:root:Trying to clean JSON file '10754_136193.json'
INFO:root:Cleaning successful


If we managed to clean the JSON file, then we can scan all the values, and create a dictionary with a [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions). 

In [7]:
row = dict(
    (ee["key"].split(".")[-1], ee["value"]) for ee in metadata
)
print(f"Author: {row['author']}")
# Cleaning the "hash" code
fields_hash = ["advisor", "author", "committeemember"] 
for ff in fields_hash:
    pos = row[ff].find(":")
    if pos > 0:
        row[ff] = row[ff][:pos]
print(f"Author: {row['author']}")

Author: Allen, Rebecca::2fd07374325b0230ae7608cdf21f8922
Author: Allen, Rebecca


In [8]:
print(row)

{'advisor': 'Sun, Shuyu', 'author': 'Allen, Rebecca', 'accessioned': '2011-07-17T09:36:20Z', 'available': '2011-07-17T09:36:20Z', 'issued': '2011-05', 'doi': '10.25781/KAUST-P8XQE', 'uri': 'http://hdl.handle.net/10754/136193', 'abstract': 'An increase in the earthâ€™s surface temperature has been directly linked to the rise of carbon dioxide (CO2) levels\\nIn the atmosphere and an enhanced greenhouse effect. CO2 sequestration is one of the proposed mitigation\\nStrategies in the effort to reduce atmospheric CO2 concentrations. Globally speaking, saline aquifers provide an adequate storage capacity for the worldâ€™s carbon emissions, and CO2 sequestration projects are currently underway in countries such as Norway, Germany, Japan, USA, and others.\\nNumerical simulators serve as predictive tools for CO2 storage, yet must model fluid transport behavior while coupling different transport processes together accurately. With regards to CO2 sequestration, an extensive amount of research has 

## Writing JSON

Finally we save our clean JSON to a file:

In [11]:
with open('metadata.json', 'w') as fout:
    JSON.dump(row, fout)